# Imports

In [2]:
import os
import warnings
warnings.filterwarnings("ignore")

from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from scipy.stats.mstats import gmean

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from sklearn.model_selection import StratifiedKFold, GridSearchCV, LeaveOneOut

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics

In [3]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 11                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data24h"
DEPRESJON_PREFIX = "manual_depresjon24h"
HYPERAKTIV_PREFIX = "manual_hyperaktiv24h"
PSYKOSE_PREFIX = "manual_psykose24h"

# Manual feature extraction

## Helper functions

In [4]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - assure format YYYY-MM-DD HH:MM:SS for "timestamp"
    - drop redundant "date" column
    - assure float32 format for "activity"
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        df["timestamp"] = pd.to_datetime(df["timestamp"],
                                         format="%Y-%m-%d %H:%M:%S")
        df.drop("date", axis=1, inplace=True)
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) &
                    (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) |
                    (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Makes sure that "timestamp" column has minute resolution with no missing
    values from start to end and replaces all NaNs in "activity" column with
    mean average value.
    
    :param data: DataFrame with "timestamp" and "activity" columns
    :returns: cleaned DataFrame
    """
    df = df.copy()  # create copy to avoid side effects
    
    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample("min", on="timestamp").mean()
    
    # recreate index and "timestamp" column
    df = df.reset_index()
    
    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())

    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """
    psd = scipy.signal.welch(
        x=df["activity"].values,
        fs=(1/11),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """
    power_spectrum = scipy.signal.welch(
        df["activity"].values,
        fs=(1/11),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [5]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [6]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [7]:
def extract_features_for_dataframes(dfs: List[pd.DataFrame], freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - nights: [21:00, 8:00)
    
    :param dfs: list of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "night", corresponding
    to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [fill_missing_activity(df) for df in full_dfs]
    full_dfs = [resample(df, freq=freq) for df in full_dfs]

    datasets = {}
    
    for part, list_of_dfs in [("night", full_dfs)]:
        features = []
        for df in list_of_dfs:
            time_features = extract_time_features(df)
            freq_features = extract_frequency_features(df)

            merged_features = pd.merge(
                time_features,
                freq_features,
                left_index=True,
                right_index=True,
                suffixes=["_time", "_freq"]
            )
            features.append(merged_features)

        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets



## Depresjon

In [8]:
dataset = Dataset(dirpath=os.path.join("data_night", "depresjon"))
condition = dataset.condition
control = dataset.control

In [9]:
condition[0]

,timestamp,date,activity
0,2003-06-20 21:00:00,2003-06-20,24
1,2003-06-20 21:01:00,2003-06-20,64
2,2003-06-20 21:02:00,2003-06-20,47
3,2003-06-20 21:03:00,2003-06-20,0
4,2003-06-20 21:04:00,2003-06-20,25
...,...,...,...
655,2003-06-21 07:55:00,2003-06-21,0
656,2003-06-21 07:56:00,2003-06-21,0
657,2003-06-21 07:57:00,2003-06-21,0
658,2003-06-21 07:58:00,2003-06-21,0


In [10]:

condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["night"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [11]:
for part, df in datasets.items():
    filename = f"{DEPRESJON_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [12]:

condition_df

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,6.250000,383.716675,100.833328,79.599998,11562.975586,2.456821,1.748712,1.016797,91.741668,79.911110,...,36417.039062,24333.826172,9.430984e+08,-1.332583,0.589108,0.843284,4.628002e+04,36417.035156,2.065168,0.625009
1,20.316668,163.466660,87.757576,84.333336,1558.604004,-0.098010,0.173928,0.428931,24.666668,86.838890,...,10460.483398,9615.545898,5.797968e+07,-0.168425,0.908459,0.727924,6.516839e+03,10460.484375,2.213524,0.727973
2,1.233333,1083.199951,210.518173,138.800003,98070.406250,3.726018,2.138875,1.418349,263.958330,136.807419,...,742928.687500,270827.062500,7.550606e+11,-0.506508,1.037575,1.169617,9.761401e+05,742928.687500,1.673742,0.421052
3,12.816667,917.833313,231.477249,25.833334,115935.023438,-0.038443,1.255166,1.402499,307.283330,179.511124,...,817749.937500,147658.062500,1.101770e+12,-1.107664,0.836612,1.283586,1.405828e+06,817749.937500,1.413001,0.266411
4,1.816667,59.049999,16.446968,5.933333,492.587708,0.036718,1.336664,1.286648,14.383333,13.338889,...,3358.645752,1200.112305,1.420884e+07,0.096646,1.296171,1.122316,3.209818e+03,3358.645752,1.827265,0.572199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,NaN
378,11.050000,191.066666,62.763638,24.733334,3811.354004,-0.528670,0.806888,0.937854,90.158334,54.253704,...,50119.691406,14485.125000,3.687559e+09,-0.834989,0.925344,1.211605,7.376414e+04,50119.691406,1.568738,0.310258
379,4.500000,157.100006,52.371212,48.799999,2133.470947,0.353883,1.024191,0.840919,56.350000,46.053703,...,14103.832031,9858.166016,1.165272e+08,-0.423864,0.965327,0.765379,1.099986e+04,14103.833008,2.193772,0.740174
380,8.083333,224.350006,68.727272,46.733334,4313.157227,0.748322,1.165695,0.911113,87.700002,58.174072,...,36915.496094,22379.478516,1.598083e+09,0.888790,1.607555,1.082907,1.430505e+04,36915.500000,1.925876,0.623274


In [13]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"depresjon_y.csv")
y.to_csv(filepath, header=False, index=False)

In [14]:
# to get ensemble:
import os

def sort_key(filename):
    parts = filename.split("_")
    # Wydobycie numerów jako liczb całkowitych
    condition_number = int(parts[1])  # Druga część np. "1" z "condition_1"
    segment_number = int(parts[3].split(".")[0])  # Trzecia część np. "1" z "segmen1.csv"
    return (condition_number, segment_number)


In [15]:
# Removing NaNs
folder_control = './data_night/depresjon/control'
folder_condition = './data_night/depresjon/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["night"].index

mask = datasets["night"].notna().all(axis=1)
datasets["night"] = datasets["night"][mask]
y = y[mask]
z_depresjon = z[mask]

## Psykose

In [16]:
dataset = Dataset(dirpath=os.path.join("data_night", "psykose"))
condition = dataset.condition
control = dataset.control

In [17]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["night"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]

    entire_df = pd.concat([condition_df, control_df], ignore_index=True)

    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [18]:
for part, df in datasets.items():
    filename = f"{PSYKOSE_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [19]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"psykose_y.csv")
y.to_csv(filepath, header=False, index=False)
#datasets["full_24h"].dropna(inplace=True)

In [20]:
# Removing NaNs
folder_control = './data_night/psykose/control'
folder_condition = './data_night/psykose/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["night"].index
mask = datasets["night"].notna().all(axis=1)
datasets["night"] = datasets["night"][mask]
y = y[mask]
z_psykose = z[mask]

Hyperactive

In [21]:
dataset = Dataset(dirpath=os.path.join("data_night", "hyperaktiv"))
condition = dataset.condition
control = dataset.control

In [22]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["night"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]

    entire_df = pd.concat([condition_df, control_df], ignore_index=True)

    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [23]:
for part, df in datasets.items():
    filename = f"{HYPERAKTIV_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [24]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"hyperaktiv_y.csv")
y.to_csv(filepath, header=False, index=False)

In [25]:
# Removing NaNs
folder_control = './data_night/hyperaktiv/control'
folder_condition = './data_night/hyperaktiv/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["night"].index
mask = datasets["night"].notna().all(axis=1)
datasets["night"] = datasets["night"][mask]
y = y[mask]
z_hyperaktiv = z[mask]

# Classification

## Classifiers, parameters, constants

In [26]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=500
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=200,
        criterion="entropy"
    ),
    "LGBM": LGBMClassifier(
        n_estimators=200,
        verbosity=-1,
        random_state=0
    ),
    "XGB": XGBClassifier(
        n_estimators=200,
        random_state=0 
    )
}


param_grids = {
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": ["balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1],
        "max_iter": [1000]
    },
    "SVM": {
        "C": [1, 10],
        "gamma": ["scale"],
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    },
    "LGBM": {
        "num_leaves": [31, 50],
        "min_child_samples": [10, 20],
        "class_weight": [None, "balanced"]
    },
    "XGB": {
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 6],
        "subsample": [0.8, 1.0],
        "colsample_bytree": [0.8, 1.0],
        "scale_pos_weight": [1, 10]
    }
}

## Classification

# Classification - Depresjon

In [26]:
dataset = DEPRESJON_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [27]:
# Assuming PROCESSED_DATA_DIR and y_filename are already defined
datasets = {}

# Load datasets
for part in ["night"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

# Sprawdzamy kształt danych po usunięciu NaN
#print(datasets['full_24h'].shape)
#print(y.shape)
#print(z_depresjon.shape)

In [28]:
print("Wyniki dla Depresjon:")
for part in ["night"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_depresjon.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Depresjon:
PART: night
  LR
    accuracy: 0.6075 +- 0.0465
    balanced_accuracy: 0.5938 +- 0.0427
    f1: 0.5208 +- 0.0415
    precision: 0.5320 +- 0.0675
    recall: 0.5126 +- 0.0182
    specificity: 0.6750 +- 0.0702
    ROC_AUC: 0.5938 +- 0.0427
    MCC: 0.1903 +- 0.0884

  SVM
    accuracy: 0.6258 +- 0.0632
    balanced_accuracy: 0.6253 +- 0.0666
    f1: 0.5791 +- 0.0805
    precision: 0.5404 +- 0.0743
    recall: 0.6257 +- 0.0943
    specificity: 0.6250 +- 0.0625
    ROC_AUC: 0.6253 +- 0.0666
    MCC: 0.2480 +- 0.1316

  RF
    accuracy: 0.5535 +- 0.0261
    balanced_accuracy: 0.4907 +- 0.0246
    f1: 0.1818 +- 0.0845
    precision: 0.3567 +- 0.0998
    recall: 0.1251 +- 0.0669
    specificity: 0.8562 +- 0.0375
    ROC_AUC: 0.4907 +- 0.0246
    MCC: -0.0345 +- 0.0760

  LGBM
    accuracy: 0.4868 +- 0.0276
    balanced_accuracy: 0.4425 +- 0.0311
    f1: 0.2294 +- 0.0474
    precision: 0.3041 +- 0.0605
    recall: 0.1851 +- 0.0398
    specificity: 0.7000 +- 0.0424
    ROC

# Classification - Psykose

In [29]:
dataset = PSYKOSE_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [30]:
for part in ["night"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

In [31]:
print("Wyniki dla Psykose:")
for part in ["night"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_psykose.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Psykose:
PART: night
  LR
    accuracy: 0.6852 +- 0.0586
    balanced_accuracy: 0.6918 +- 0.0644
    f1: 0.6479 +- 0.0847
    precision: 0.5933 +- 0.0555
    recall: 0.7273 +- 0.1379
    specificity: 0.6562 +- 0.0884
    ROC_AUC: 0.6918 +- 0.0644
    MCC: 0.3810 +- 0.1221

  SVM
    accuracy: 0.7259 +- 0.0378
    balanced_accuracy: 0.6835 +- 0.0380
    f1: 0.5670 +- 0.0826
    precision: 0.8260 +- 0.1256
    recall: 0.4545 +- 0.1185
    specificity: 0.9125 +- 0.0996
    ROC_AUC: 0.6835 +- 0.0380
    MCC: 0.4413 +- 0.0867

  RF
    accuracy: 0.7148 +- 0.0222
    balanced_accuracy: 0.6557 +- 0.0215
    f1: 0.4894 +- 0.0402
    precision: 0.9205 +- 0.1080
    recall: 0.3364 +- 0.0364
    specificity: 0.9750 +- 0.0364
    ROC_AUC: 0.6557 +- 0.0215
    MCC: 0.4317 +- 0.0662

  LGBM
    accuracy: 0.7074 +- 0.0319
    balanced_accuracy: 0.6551 +- 0.0350
    f1: 0.5081 +- 0.0620
    precision: 0.8010 +- 0.0674
    recall: 0.3727 +- 0.0530
    specificity: 0.9375 +- 0.0198
    ROC_AU

# Classification - Hyperactive

In [27]:
dataset = HYPERAKTIV_PREFIX
y_filename = "hyperaktiv_y.csv"

In [28]:
for part in ["night"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

In [29]:
print("Wyniki dla Hyperaktiv:")
for part in ["night"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            #X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_hyperaktiv.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Hyperaktiv:
PART: night
  LR


/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3

    accuracy: 0.5241 +- 0.0198
    balanced_accuracy: 0.5352 +- 0.0172
    f1: 0.4202 +- 0.0266
    precision: 0.5859 +- 0.0240
    recall: 0.3283 +- 0.0286
    specificity: 0.7421 +- 0.0242
    ROC_AUC: 0.5352 +- 0.0172
    MCC: 0.0771 +- 0.0370

  SVM
    accuracy: 0.5736 +- 0.0183
    balanced_accuracy: 0.5833 +- 0.0203
    f1: 0.5073 +- 0.0385
    precision: 0.6544 +- 0.0487
    recall: 0.4218 +- 0.0667
    specificity: 0.7448 +- 0.0864
    ROC_AUC: 0.5833 +- 0.0203
    MCC: 0.1783 +- 0.0451

  RF
    accuracy: 0.5272 +- 0.0151
    balanced_accuracy: 0.5357 +- 0.0159
    f1: 0.4611 +- 0.0210
    precision: 0.5779 +- 0.0295
    recall: 0.3853 +- 0.0309
    specificity: 0.6860 +- 0.0413
    ROC_AUC: 0.5357 +- 0.0159
    MCC: 0.0750 +- 0.0339

  LGBM
    accuracy: 0.5549 +- 0.0313
    balanced_accuracy: 0.5608 +- 0.0291
    f1: 0.5154 +- 0.0396
    precision: 0.6025 +- 0.0301
    recall: 0.4511 +- 0.0453
    specificity: 0.6704 +- 0.0256
    ROC_AUC: 0.5608 +- 0.0291
    MCC: 0.1240 +